In [1]:
import os
import warnings
warnings.simplefilter('ignore')

import geopandas as gpd
import pandas as pd
import numpy as np

### Define exposure function

In [2]:
def func_exposure(hazard, census, var, threshold):
    
    '''
    hazard : hazard aggregated to census tract or similar unit
    census : census tract containings the socio-economic variables
    var : the name of column name of variable to assessed from the hazard file
    threshold: value of threshod for the input variable of hazard file
    '''
    
    exposed_area = hazard[hazard[var] >= threshold]
    exposed_area = exposed_area[['code_tract', var]]
    exposed_pop = exposed_area.merge(census, on  = 'code_tract', how = 'left')
    exposed_tract = census[['code_tract', 'geometry']].merge(exposed_pop.drop('geometry', axis = 1), on  = 'code_tract', how = 'left')
    
    for k in keep[:-3]:
        exposed_tract['prop3_'+k] = round(100*exposed_tract[k]/exposed_tract[k].sum(),4)
    
    exposed_df = exposed_tract[keep[:-1]]
    #variable = []
    quantity = []
    share_tract = []
    share_city = []
    for i in keep[:-3]:
        #variable.append(i)
        quantity.append(exposed_df[i].sum())
        share_tract.append(round(100*exposed_df[i].sum()/exposed_df['p_tot'].sum(),4))
        share_city.append(round(100*exposed_df[i].sum()/census[i].sum(),4))
    #variable.append('p_mean_inc1')
    quantity.append((exposed_df['p_mean_inc1']*exposed_df['p_10+yr']).sum()/exposed_df['p_10+yr'].sum())
    share_tract.append(100)
    share_city.append(round(100*((exposed_df['p_mean_inc1']*exposed_df['p_10+yr']).sum()/exposed_df['p_10+yr'].sum()/census['p_mean_inc1_city'].mean()), 4))
    
    exposed_stats = pd.DataFrame()
    #exposed_stats['variable'] = variable
    exposed_stats['Name'] = ['Total Population', 'Female Population', 'Male Population',
                             'White Population', 'Black Population', 'Yellow Population', 'Brown Population', 'Indigenous Population',
                             'Population with Age below 14', 'Population with Age between 15 and 19', 'Population with Age between 20 and 59', 'Population with Age above 60',
                             'Mean per capita Income']
    exposed_stats['Value'] = quantity
    exposed_stats['Share in Exposed Population'] = share_tract
    exposed_stats['Share in City'] = share_city
    
    return exposed_tract, exposed_stats

### Get data from s3

In [3]:
#hazard_grid = gpd.read_file('https://cities-socio-economic-vulnerability.s3.eu-west-3.amazonaws.com/case-studies/campinas/maps/all_hazard_grid.geojson')

hazard_tract = gpd.read_file('https://cities-socio-economic-vulnerability.s3.eu-west-3.amazonaws.com/case-studies/campinas/maps/all_hazard_tract.geojson')

census_tract = gpd.read_file('https://cities-socio-economic-vulnerability.s3.eu-west-3.amazonaws.com/case-studies/campinas/maps/census_tract_keep.geojson')

In [4]:
#socioeconomic parameters - columns to keep
keep = ['p_tot', 'p_tot_f', 'p_tot_m'] + ['p_wht', 'p_black', 'p_yellow', 'p_brown', 'p_indig'] + ['p_0to14yr', 'p_15to19yr', 'p_20to59yr', 'p_60+yr'] + ['p_mean_inc1', 'p_10+yr', 'p_mean_inc1_city']

In [5]:
hazard_tract.describe() #get stats to decide the threshold. Currently using 75% i.e. last quartile

,Heat_T_mean_max_max,Heat_T_mean_max_mean,Heat_T_mean_mean_max,Heat_T_mean_mean_mean,Heat_indice_max,Heat_indice_mean,Heat_vuln_indic_max,Heat_vuln_indic_mean,Heat_risco_indi_max,Heat_risco_indi_mean,Landslide_indice_sum_max,Landslide_indice_sum_mean,Flood_indice_sum_max,Flood_indice_sum_mean,Inundation_indice_sum_max,Inundation_indice_sum_mean
count,1738.000000,1738.000000,1738.000000,1738.000000,1738.000000,1738.000000,1736.000000,1738.000000,1736.000000,1738.000000,1749.000000,1749.000000,1749.000000,1749.000000,1749.000000,1749.000000
mean,32.604686,31.309942,31.494939,26.593464,4.733236,3.996608,1.647004,1.477227,6.306415,5.443036,3.404741,2.221578,4.135384,2.699072,2.246408,1.195017
std,0.550352,2.298860,1.700467,3.463774,0.255564,0.520559,0.472460,0.351274,0.532868,0.742559,1.347298,0.951442,1.166247,0.895464,1.184680,0.658361
min,17.238000,16.938740,16.182750,13.500080,2.432000,2.028800,0.337000,0.000000,3.338000,0.000000,0.826000,0.027960,0.938000,0.167940,0.175000,0.054620
25%,32.323000,31.757445,31.773355,24.645183,4.775000,3.703958,1.449000,1.360927,6.153500,5.120040,2.386000,1.570760,3.328000,2.140460,1.316000,0.716330
50%,32.323000,32.209765,32.056250,27.318225,4.818000,4.105560,1.527000,1.476855,6.299000,5.576905,3.113000,2.000000,4.098000,2.704420,1.984000,1.051830
75%,33.165000,32.280173,32.190875,29.179453,4.838000,4.385335,1.587000,1.542555,6.379000,5.883443,4.287000,2.585350,4.899000,3.251750,2.982000,1.515200
max,33.270000,33.261690,33.270000,32.323000,5.000000,4.858000,3.241000,3.219630,8.241000,7.931000,7.412000,5.795940,7.895000,5.874750,6.645000,4.622740


### Run thresholding function

In [6]:
heat_exposed_tract, heat_exposed_stats = func_exposure(hazard_tract, census_tract, 'Heat_T_mean_mean_mean', 29.179453)

In [7]:
flood_exposed_tract, flood_exposed_stats= func_exposure(hazard_tract, census_tract, 'Flood_indice_sum_mean', 3.251750)

In [8]:
landslide_exposed_tract, landslide_exposed_stats = func_exposure(hazard_tract, census_tract, 'Landslide_indice_sum_mean', 2.585350)

In [9]:
inundation_exposed_tract, inundation_exposed_stats = func_exposure(hazard_tract, census_tract, 'Inundation_indice_sum_mean', 1.515200)

### Save

In [10]:
heat_exposed_tract.to_file('../Data/heat_exposed75_tract.geojson')

heat_exposed_stats.to_excel('../Data/heat_exposed75_stats.xlsx', index = False)

In [11]:
flood_exposed_tract.to_file('../Data/flood_exposed75_tract.geojson')

flood_exposed_stats.to_excel('../Data/flood_exposed75_stats.xlsx', index = False)

In [12]:
landslide_exposed_tract.to_file('../Data/landslide_exposed75_tract.geojson')

landslide_exposed_stats.to_excel('../Data/landslide_exposed75_stats.xlsx', index = False)

In [13]:
inundation_exposed_tract.to_file('../Data/inundation_exposed75_tract.geojson')

inundation_exposed_stats.to_excel('../Data/inundation_exposed75_stats.xlsx', index = False)